In [6]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import pandas as pd
import torch
from datetime import datetime, timezone
import logging

logging.basicConfig(
    level=logging.INFO,  # Set the logging level
    format='%(asctime)s - %(levelname)s - %(message)s',  # Format for the log messages
    handlers=[
        logging.StreamHandler()  # Log to the console
    ]
)

%reload_ext autoreload
%autoreload 2
from core_data_prep.core_data_prep import DataPreparer
from core_data_prep.validations import Validator

from data.raw.retrievers.alpaca_markets_retriever import AlpacaMarketsRetriever
from data.raw.retrievers.stooq_retriever import StooqRetriever
from config.constants import *
from data.processed.dataset_creation import DatasetCreator
from data.processed.indicators import *
from data.processed.targets import Balanced3ClassClassification
from data.processed.normalization import ZScoreOverWindowNormalizer, ZScoreNormalizer, MinMaxNormalizer
from data.processed.dataset_pytorch import DatasetPytorch
from modeling.trainer import Trainer
from modeling.evaluate import evaluate_lgb_regressor, evaluate_torch_regressor, evaluate_torch_regressor_multiasset
from modeling.modeling_utils import print_model_parameters

from modeling.rl.environment import PortfolioEnvironment
from modeling.rl.state import State
from modeling.rl.agent import RlAgent
from modeling.rl.algorithms.policy_gradient import PolicyGradient
from modeling.rl.actors.actor import RlActor
from modeling.rl.actors.signal_predictor_actor import SignalPredictorActor
from modeling.rl.actors.high_energy_low_friction_actor import HighEnergyLowFrictionActor
from modeling.rl.actors.xsmom_actor import XSMomActor
from modeling.rl.actors.tsmom_actor import TSMomActor
from modeling.rl.actors.blsw_actor import BLSWActor
from modeling.rl.actors.allocation_propogation_actor import AllocationPropogationActor
from modeling.rl.actors.market_actor import MarketActor
from modeling.rl.trajectory_dataset import TrajectoryDataset
from modeling.rl.metrics import MetricsCalculator, DEFAULT_METRICS
from modeling.rl.reward import EstimatedReturnReward
from modeling.rl.loss import SumLogReturnLoss, ReinforceLoss
from modeling.rl.visualization.wealth_plot import plot_cumulative_wealth
from modeling.rl.visualization.position_plot import plot_position_heatmap
from config.experiments.cur_experiment import config

torch.backends.cudnn.benchmark = config.train_config.cudnn_benchmark

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cuda')

In [28]:
retriever = config.data_config.retriever

In [ ]:
result = retriever.latest_bars(config.data_config.symbol_or_symbols, limit=100)

AssertionError: Not enough bars for pulled for AAPL to satisfy the limit of 10000; pulled 3 days and got 2009 bars.

In [32]:
result['AAPL']

,date,open,high,low,close,volume,vwap
0,2025-12-03 15:22:00-05:00,284.3600,284.4699,284.3100,284.425,47572.0,284.393164
1,2025-12-03 15:23:00-05:00,284.4350,284.4564,284.3600,284.440,43799.0,284.394887
2,2025-12-03 15:24:00-05:00,284.4400,284.5100,284.4300,284.460,56850.0,284.462945
3,2025-12-03 15:25:00-05:00,284.4445,284.4900,284.3900,284.420,39544.0,284.450113
4,2025-12-03 15:26:00-05:00,284.4400,284.5099,284.3600,284.400,46583.0,284.434670
...,...,...,...,...,...,...,...
995,2025-12-05 09:57:00-05:00,280.1600,280.2400,280.1400,280.198,61359.0,280.188009
996,2025-12-05 09:58:00-05:00,280.1800,280.2600,280.1400,280.140,43272.0,280.196574
997,2025-12-05 09:59:00-05:00,280.1683,280.2300,280.0306,280.180,64635.0,280.151694
998,2025-12-05 10:00:00-05:00,280.1600,280.4050,280.0501,280.159,76739.0,280.266669


In [37]:
from data.processed.data_processing_utils import convert_time_to_eastern
from datetime import datetime


datetime.now(timezone.utc)

datetime.datetime(2025, 12, 5, 15, 20, 31, 23930, tzinfo=datetime.timezone.utc)

In [38]:
convert_time_to_eastern(datetime.now(timezone.utc))

Timestamp('2025-12-05 10:20:32.192092-0500', tz='America/New_York')

In [11]:
prediction = np.ones(len(config.data_config.symbol_or_symbols))
cur_allocation = {symbol: prediction[i] - 1.5 for i, symbol in enumerate(config.data_config.symbol_or_symbols)}
available_cash = 10000
import time

In [12]:
start_time = time.time()

allocation = {symbol: prediction[i] for i, symbol in enumerate(config.data_config.symbol_or_symbols)}

allocation_difference = {symbol: allocation[symbol] - cur_allocation[symbol] for symbol in allocation}

buy_positions = {symbol: allocation_difference[symbol] for symbol in allocation_difference if allocation_difference[symbol] > 0}
sell_positions = {symbol: allocation_difference[symbol] for symbol in allocation_difference if allocation_difference[symbol] < 0}

buy_cash_percentage = sum(buy_positions.values())
buy_cash_per_asset = {symbol: buy_positions[symbol] * available_cash * buy_cash_percentage for symbol in buy_positions}

end_time = time.time()
print(f"Time taken: {end_time - start_time} seconds")

Time taken: 0.0010018348693847656 seconds


In [15]:
from core_inference.brokerage_proxies.alpaca_brokerage_proxy import AlpacaBrokerageProxy

alpaca_brokerage_proxy = AlpacaBrokerageProxy()

2025-12-06 19:18:45,297 - INFO - AlpacaBrokerageProxy instantiated. Available cash: 99996.75. Open positions:


In [7]:
from config.constants import Constants

all_djia_stock = set(Constants.Data.DJIA_2000_2004 + Constants.Data.DJIA_2004_2008 \
    + Constants.Data.DJIA_2008_2008 + Constants.Data.DJIA_2008_2009 \
    + Constants.Data.DJIA_2009_2012 + Constants.Data.DJIA_2012_2013 \
    + Constants.Data.DJIA_2013_2015 + Constants.Data.DJIA_2015_2017 \
    + Constants.Data.DJIA_2017_2018 + Constants.Data.DJIA_2018_2019)

len(all_djia_stock)

44

In [12]:
len(os.listdir("..\\data\\raw\\stooq\\bars\\us_all\\data\\daily\\us"))

8423

In [16]:
root_dir = "..\\data\\raw\\stooq\\bars\\us_all\\data\\daily\\us"
output_dir = "..\\data\\raw\\stooq\\bars\\djia_all"
available_stocks = [name.split('.')[0] for name in os.listdir(root_dir)]
for stock in all_djia_stock:
    if stock.lower() not in available_stocks:
        print(f'Skipping {stock} because it is not available')
    else: 
        import shutil
        shutil.copy2(os.path.join(root_dir, f'{stock.lower()}.us.txt'), os.path.join(output_dir, f'{stock.lower()}_us_d.txt'))

Skipping EK because it is not available
